## Model Selection Demo:

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from itertools import combinations

In [2]:
# Simulate data
np.random.seed(0)
n = 1000
p = 5
X = np.random.randn(n, p)
beta = np.array([3, 2, 0, 0, 0])  # Only two variables are nonzero
y = X @ beta + np.random.randn(n) * 0.5

# Add a constant to X for intercept
X = sm.add_constant(X)

In [3]:
# Best subset selection
def best_subset_selection(X, y):
    n, p = X.shape
    models = []
    
    for k in range(1, p + 1):  # Iterate over subset sizes
        for combo in combinations(range(1, p), k):  # Generate combinations of predictors
            combo = (0,) + combo  # Include the intercept
            X_subset = X[:, combo]
            model = sm.OLS(y, X_subset).fit()
            models.append((model, combo))
    
    return models


In [11]:

# Calculate metrics
def calculate_metrics(model, X, y):
    n = len(y)
    k = model.df_model  # Number of predictors, excluding intercept


    
    # AIC
    aic = model.aic
    
    # BIC
    bic = model.bic
    
    
    # PRESS (Prediction Sum of Squares)
    hat_matrix = X @ np.linalg.inv(X.T @ X) @ X.T
    residuals = model.resid
    press = np.sum((residuals / (1 - np.diag(hat_matrix))) ** 2)
    
    # Adjusted R-squared
    r2 = model.rsquared
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - k)

    #Mallow CP
    rss = np.sum(residuals ** 2)
    variance = np.var(y, ddof=1)
    cp = rss / variance - (n - 2 * (k + 1))
    
    return aic, bic, press, adj_r2, cp, int(k) #dont consider intercept as a predictor



In [12]:
# Run best subset selection
models = best_subset_selection(X, y)

In [13]:
# Store results in pd DataFrame
results = []
for model, combo in models:
    aic, bic, press, adj_r2, cp, num_predictors = calculate_metrics(model, X[:, combo], y)
    results.append({
        'Predictors': combo,
        'n_Predictors': num_predictors,
        'AIC': aic,
        'BIC': bic,
        'PRESS': press,
        'Adjusted R^2': adj_r2,
        'Mallows CP': cp
    })

# Convert results to pd DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='n_Predictors').reset_index(drop=True)


In [14]:
# Display our results
pd.set_option('display.max_columns', None)  # Show all columns
results_df #0 represents intercept – all models include intercept

Predictors  n_Predictors          AIC          BIC         PRESS  \
0               (0, 1)             1  4168.840872  4178.656382   3785.152258   
1               (0, 2)             1  5038.813069  5048.628579   9034.712100   
2               (0, 3)             1  5398.209642  5408.025152  12939.470547   
3               (0, 4)             1  5397.641492  5407.457002  12933.138971   
4               (0, 5)             1  5398.175148  5407.990659  12939.770161   
5            (0, 4, 5)             2  5399.000246  5413.723512  12950.664816   
6            (0, 3, 5)             2  5399.630015  5414.353281  12957.963976   
7            (0, 3, 4)             2  5399.106491  5413.829757  12951.514629   
8            (0, 2, 5)             2  5039.863992  5054.587258   9043.968854   
9            (0, 2, 4)             2  5039.363033  5054.086299   9040.554097   
10           (0, 2, 3)             2  5040.795516  5055.518782   9051.813650   
11           (0, 1, 5)             2  4170.835013  4185.558279   3792.691273   
12           (0, 1, 4)             2  4170.831107  4185.554373   3793.241268   
13           (0, 1, 3)             2  4167.760183  4182.483448   3780.874128   
14           (0, 1, 2)             2  1424.115862  1438.839128    243.221770   
15        (0, 3, 4, 5)             3  5400.451808  5420.082829  12968.914241   
16        (0, 2, 4, 5)             3  5040.303544  5059.934565   9048.707510   
17        (0, 2, 3, 5)             3  5041.849107  5061.480128   9061.154207   
18        (0, 2, 3, 4)             3  5041.345866  5060.976887   9057.644658   
19        (0, 1, 4, 5)             3  4172.824529  4192.455550   3800.766596   
20        (0, 1, 2, 3)             3  1425.005414  1444.636436    243.514714   
21        (0, 1, 3, 4)             3  4169.750085  4189.381106   3789.024898   
22        (0, 1, 2, 5)             3  1425.430861  1445.061882    243.507077   
23        (0, 1, 2, 4)             3  1426.061285  1445.692306    243.710573   
24        (0, 1, 3, 5)             3  4169.751048  4189.382069   3788.374057   
25     (0, 2, 3, 4, 5)             4  5042.289172  5066.827948   9065.882384   
26     (0, 1, 2, 3, 4)             4  1426.950318  1451.489094    244.003823   
27     (0, 1, 2, 3, 5)             4  1426.300991  1450.839767    243.798024   
28     (0, 1, 2, 4, 5)             4  1427.357141  1451.895918    243.996967   
29     (0, 1, 3, 4, 5)             4  4171.740037  4196.278814   3796.512782   
30  (0, 1, 2, 3, 4, 5)             5  1428.226372  1457.672904    244.288229   

    Adjusted R^2  Mallows CP  
0       0.707679 -703.970918  
1       0.302274 -298.971995  
2       0.000532    2.468058  
3       0.001100    1.900939  
4       0.000567    2.433618  
5       0.000740    3.261245  
6       0.000111    3.889486  
7       0.000634    3.367204  
8       0.302238 -297.633214  
9       0.302587 -297.981978  
10      0.301587 -296.984230  
11      0.707387 -701.972629  
12      0.707389 -701.973770  
13      0.708286 -702.869185  
14      0.981232 -975.269965  
15      0.000286    4.714459  
16      0.302627 -296.719011  
17      0.301548 -295.643580  
18      0.301900 -295.993927  
19      0.707097 -699.975691  
20      0.981234 -973.290752  
21      0.707996 -700.872124  
22      0.981226 -973.282791  
23      0.981215 -973.270987  
24      0.707996 -700.871844  
25      0.301937 -294.729004  
26      0.981217 -971.291783  
27      0.981229 -971.303927  
28      0.981209 -971.284171  
29      0.707706 -698.875050  
30      0.981211 -969.305322